# Support Vector Regression - Experiment

This is a component that trains a Support Vector Regression model using [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html). 
<br>
Scikit-learn is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection and evaluation, and many other utilities.

This notebook shows:
- how to use the [SDK](https://platiagro.github.io/sdk/) to load datasets, save models and other artifacts.
- how to declare parameters and use them to build reusable components.

## Declare parameters and model hyperparameters
Components may declare (and use) these default parameters:
- dataset
- target

Use these parameters to load/save datasets, models, metrics, and figures with the help of [PlatIAgro SDK](https://platiagro.github.io/sdk/). <br />
You may also declare custom parameters to set when running an experiment.

Select the hyperparameters and their respective values to be used when training the model:
- kernel
- degree
- gamma
- C
- max_iter

These parameters are just a few offered by the model class, you may also use another existing parameter. <br />
Check the [model parameters](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR) for more information.

In [ ]:
# parameters
dataset = "boston" #@param {type:"string"}
target = "medv" #@param {type:"feature", label:"Atributo alvo", description: "Seu modelo será treinado para prever os valores do alvo."}

# features to apply One Hot Encoder
one_hot_features = "" #@param {type:"feature",label:"Features para fazer codificação one-hot", description: "Seu modelo utilizará a codificação one-hot para as features selecionadas. As demais features categóricas serão codificadas utilizando a codificação ordinal."}

# hyperparameters
kernel = "rbf" #@param ["linear", "poly", "rbf", "sigmoid", "precomputed"] {type:"string", label:"Kernel", description:"Especifica o tipo de kernel a ser usado no algoritmo"}
degree = 3 #@param {type:"integer", label:"Grau", description:"Grau da função polinomial do kernel ('poly'). Ignorado por outros kernels"}
gamma = "auto" #@param ["scale", "auto"] {type: "string", label:"Gama", description:"Coeficiente de kernel para 'rbf', 'poly' e 'sigmoid'"} 
C = 1.0 #@param {type:"number", label:"Regularização", description:"A força da regularização é inversamente proporcional a C. Deve ser positivo. Penalidade é l2²"}
max_iter = -1 #@param {type:"integer", label:"Iteração", description:"Limite fixo nas iterações no solver, ou -1 sem limite"}

## Load dataset

Import and put the whole dataset in a pandas.DataFrame.

In [ ]:
from platiagro import load_dataset

df = load_dataset(name=dataset)
X = df.drop(target, axis=1).to_numpy()
y = df[target].to_numpy()

## Load metadata about the dataset
For example, below we get the feature type for each column in the dataset. (eg. categorical, numerical, or datetime)

In [ ]:
import numpy as np
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]

columns = df.columns.to_numpy()
featuretypes = np.array(featuretypes)
target_index = np.argwhere(columns == target)
columns = np.delete(columns, target_index)
featuretypes = np.delete(featuretypes, target_index)

## Split dataset into train/test splits

Training Dataset: the sample of data used to fit the model.

Test Dataset: the sample of data used to provide an unbiased evaluation of a model fit on the training dataset.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,  train_size=0.7)

## Features configuration

In [ ]:
from platiagro.featuretypes import NUMERICAL

# Selects the indexes of numerical and non-numerical features
numerical_indexes = np.where(featuretypes == NUMERICAL)[0]
non_numerical_indexes = np.where(~(featuretypes == NUMERICAL))[0]


# Selects non-numerical features to apply ordinal encoder or one-hot encoder
one_hot_features = np.asarray(one_hot_features)
non_numerical_indexes_one_hot = np.where(~(featuretypes == NUMERICAL) & np.isin(columns,one_hot_features))[0]
non_numerical_indexes_ordinal = np.where(~(featuretypes == NUMERICAL) & ~(np.isin(columns,one_hot_features)))[0]


# After the step handle_missing_values, 
# numerical features are grouped in the beggining of the array
numerical_indexes_after_handle_missing_values = \
    np.arange(len(numerical_indexes))
non_numerical_indexes_after_handle_missing_values = \
    np.arange(len(numerical_indexes), len(featuretypes))
non_numerical_indexes_after_handle_missing_values_one_hot = non_numerical_indexes_after_handle_missing_values[np.where(np.isin(non_numerical_indexes,non_numerical_indexes_one_hot))[0]]         
non_numerical_indexes_after_handle_missing_values_ordinal = non_numerical_indexes_after_handle_missing_values[np.where(np.isin(non_numerical_indexes,non_numerical_indexes_ordinal))[0]]                                              

## Fit a model using sklearn.svm.SVR

In [ ]:
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.one_hot import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[
    ('handle_missing_values',
     ColumnTransformer(
        [('imputer_mean', SimpleImputer(strategy='mean'), numerical_indexes),
         ('imputer_mode', SimpleImputer(strategy='most_frequent'), non_numerical_indexes)],
         remainder='drop')),
    ('handle_categorical_features',
    ColumnTransformer(
         [('feature_encoder_ordinal', OrdinalEncoder(), non_numerical_indexes_after_handle_missing_values_ordinal),
          ('feature_encoder_onehot', OneHotEncoder(), non_numerical_indexes_after_handle_missing_values_one_hot)],
         remainder='passthrough')),
    ('estimator', SVR(kernel=kernel,
                      degree=degree,
                      gamma=gamma,
                      C=C,
                      max_iter=max_iter))
])

pipeline.fit(X_train, y_train)    

## Measure the performance

R² corresponds to the squared correlation between the observed outcome values and the predicted values by the model.

In [ ]:
from sklearn.metrics import r2_score

# uses the model to make predictions on the Test Dataset
y_pred = pipeline.predict(X_test)

# computes R²
r2 = r2_score(y_test, y_pred)
pipeline.fit(X, y)

## Save metrics

Record the metrics used to evaluate the model.<br>
It's a good way to document the experiments, and also help to avoid running the same experiment twice. 

In [ ]:
from platiagro import save_metrics

save_metrics(r2_score=r2)

## Save figure

Record a matplotlib figure to document the experiment.

In [ ]:
import matplotlib.pyplot as plt
from platiagro import save_figure
from scipy.stats import gaussian_kde


def annotate_plot(e, s, plt, y_lim, h, abs_err):
    if h < 2:
        p = 0.05
    else:
        p = 0.1
    plt.annotate("", xy=(max(e), y_lim[1]/h),
                 xytext=(0, y_lim[1]/h),
                 arrowprops=dict(arrowstyle="->"))
    plt.annotate("", xy=(min(e), y_lim[1]/h),
                 xytext=(0, y_lim[1]/h),
                 arrowprops=dict(arrowstyle="->"))
    plt.annotate("{}%".format(s),
                 xy=(0, (1+p)*y_lim[1]/h),
                 ha="center")
    if abs_err:
        plt.annotate("{:.2f}".format(max(e)),
                     xy=((0+max(e))/2, (1-p)*y_lim[1]/h),
                     ha="center")
        plt.annotate("{:.2f}".format(min(e)),
                     xy=((0+min(e))/2, (1-p)*y_lim[1]/h),
                     ha="center")
    else:
        plt.annotate("{:.2f}%".format(100*max(e)),
                     xy=((0+max(e))/2, (1-p)*y_lim[1]/h),
                     ha="center")
        plt.annotate("{:.2f}%".format(100*min(e)),
                     xy=((0+min(e))/2, (1-p)*y_lim[1]/h),
                     ha="center")

In [ ]:
abs_err = False
if any(y_test==0):
    err = y_pred - y_test
    abs_err = True
else:
    err = (y_pred - y_test)/y_test

sorted_idx = np.argsort(np.abs(err))
n = int(0.7*len(y_test))
idx = sorted_idx[:n]
e = err[idx]

n = int(0.95*len(y_test))
idx = sorted_idx[:n]
aux = err[idx]
x_lim = (aux.min(), aux.max())

plt.figure()

kde = gaussian_kde(err)
x_err = np.linspace(err.min(), err.max(), 1000)
p_err = kde(x_err)
plt.plot(x_err, p_err, 'b-')

y_lim = plt.ylim()
plt.ylim((0, y_lim[1]))
y_lim = plt.ylim()
plt.xlim(x_lim)
plt.plot([e.min(), e.min()], y_lim, "r--")
plt.plot([e.max(), e.max()], y_lim, "r--")

# Shade the area between e.min() and e.max()
plt.fill_betweenx(y_lim, e.min(), e.max(),
                  facecolor="red",  # The fill color
                  color="red",      # The outline color
                  alpha=0.2)        # Transparency of the fill

annotate_plot(e, 70, plt, y_lim, 2, abs_err)
annotate_plot(aux, 95, plt, y_lim, 1.2, abs_err)

plt.grid(True)
plt.title("Error Distribution")

save_figure(figure=plt.gcf())

## Save model

Stores the model artifacts in a object storage.<br>
It will make the model available for future deployments.

In [ ]:
from platiagro import save_model

save_model(pipeline=pipeline, columns=columns)